# Modelagem Quântica da Molécula de H₂

Este notebook implementa a modelagem da molécula de H₂ utilizando **PennyLane** e **AWS Braket**. 
Ele realiza a simulação quântica da molécula de hidrogênio utilizando o **Variational Quantum Eigensolver (VQE)** para determinar a energia do estado fundamental. 

## Parâmetros do Modelo:

- **basis** (*str*): Base utilizada para representar a função de onda molecular. Exemplo: "sto-3g".
- **ansatz** (*str*): Tipo de circuito quântico usado para representar o estado quântico. Exemplo: "EfficientSU2".
- **backend** (*str*): Define onde a simulação será executada ("local" para simulador ou "aws" para rodar na AWS Braket).
- **shots** (*int*): Número de amostragens para a execução do circuito.
- **optimizer** (*str*): Algoritmo utilizado para otimizar os parâmetros do circuito ("GradientDescent", "COBYLA", "Adam", "Nelder-Mead").
- **fit** (*str*): Método de ajuste dos resultados (exemplo: "quadrático", "polinomial").
- **mapper** (*str*): Tipo de mapeamento de férmions para qubits (exemplo: "Jordan-Wigner", "Bravyi-Kitaev").

In [ ]:
import os
import numpy as np
import pennylane as qml
from braket.aws import AwsDevice
from braket.pennylane_plugin import BraketDevice

class H2Model:
    def __init__(self, basis="sto-3g", ansatz="EfficientSU2", backend="local", 
                 shots=1000, optimizer="COBYLA", fit="quadratic", mapper="Jordan-Wigner"):
        self.basis = basis
        self.ansatz = ansatz
        self.backend = backend
        self.shots = shots
        self.optimizer = optimizer
        self.fit = fit
        self.mapper = mapper
        self.device = self.setup_device()
        self.H = self.generate_hamiltonian()

    def setup_device(self):
        if self.backend == "aws":
            return BraketDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
        else:
            return qml.device("default.qubit", wires=4)

    def generate_hamiltonian(self):
        coeffs = [-0.242742, 0.177712, 0.177712, 0.122933, 0.122933, 0.167683, 0.167683]
        ops = [
            qml.Identity(wires=0),
            qml.PauliZ(wires=0) @ qml.PauliZ(wires=1),
            qml.PauliZ(wires=0) @ qml.PauliZ(wires=2),
            qml.PauliZ(wires=0) @ qml.PauliZ(wires=3),
            qml.PauliZ(wires=1) @ qml.PauliZ(wires=2),
            qml.PauliZ(wires=1) @ qml.PauliZ(wires=3),
            qml.PauliZ(wires=2) @ qml.PauliZ(wires=3)
        ]
        return qml.Hamiltonian(coeffs, ops)

    def run_vqe(self):
        @qml.qnode(self.device)
        def circuit(params):
            qml.BasisState(np.array([1, 1, 0, 0]), wires=[0, 1, 2, 3])
            qml.RY(params[0], wires=0)
            qml.RY(params[1], wires=1)
            qml.RY(params[2], wires=2)
            qml.RY(params[3], wires=3)
            qml.CNOT(wires=[0, 1])
            qml.CNOT(wires=[1, 2])
            qml.CNOT(wires=[2, 3])
            return qml.expval(self.H)
        
        opt = qml.GradientDescentOptimizer(stepsize=0.4)
        params = np.random.randn(4, requires_grad=True)
        for i in range(100):
            params = opt.step(lambda p: circuit(p), params)
            energy = circuit(params)
            if i % 10 == 0:
                print(f"Step {i}: Energy = {energy}")
        return energy

# Executar o modelo para H2
test_H2 = H2Model()
energy_H2 = test_H2.run_vqe()
print(f"Energia do estado fundamental do H2: {energy_H2:.6f} Ha")

ModuleNotFoundError: No module named 'pennylane'

In [ ]:
import os
import numpy as np
import pennylane as qml
from braket.aws import AwsDevice
from braket.pennylane_plugin import BraketDevice

class H2Model:
    def __init__(self, basis="sto-3g", ansatz="EfficientSU2", backend="local", 
                 shots=1000, optimizer="COBYLA", fit="quadratic", mapper="Jordan-Wigner"):
        self.basis = basis
        self.ansatz = ansatz
        self.backend = backend
        self.shots = shots
        self.optimizer = optimizer
        self.fit = fit
        self.mapper = mapper
        self.device = self.setup_device()
        self.H = self.generate_hamiltonian()

    def setup_device(self):
        if self.backend == "aws":
            return BraketDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
        else:
            return qml.device("default.qubit", wires=4)

    def generate_hamiltonian(self):
        coeffs = [-0.242742, 0.177712, 0.177712, 0.122933, 0.122933, 0.167683, 0.167683]
        ops = [
            qml.Identity(wires=0),
            qml.PauliZ(wires=0) @ qml.PauliZ(wires=1),
            qml.PauliZ(wires=0) @ qml.PauliZ(wires=2),
            qml.PauliZ(wires=0) @ qml.PauliZ(wires=3),
            qml.PauliZ(wires=1) @ qml.PauliZ(wires=2),
            qml.PauliZ(wires=1) @ qml.PauliZ(wires=3),
            qml.PauliZ(wires=2) @ qml.PauliZ(wires=3)
        ]
        return qml.Hamiltonian(coeffs, ops)

    def get_optimizer(self):
        optimizers = {
            "GradientDescent": qml.GradientDescentOptimizer(stepsize=0.4),
            "COBYLA": qml.optimize.COBYLAOptimizer(),
            "Adam": qml.AdamOptimizer(stepsize=0.1),
            "Nelder-Mead": qml.optimize.NelderMeadOptimizer(),
        }
        return optimizers.get(self.optimizer, qml.GradientDescentOptimizer(stepsize=0.4))

    def run_vqe(self):
        @qml.qnode(self.device)
        def circuit(params):
            qml.BasisState(np.array([1, 1, 0, 0]), wires=[0, 1, 2, 3])
            qml.RY(params[0], wires=0)
            qml.RY(params[1], wires=1)
            qml.RY(params[2], wires=2)
            qml.RY(params[3], wires=3)
            qml.CNOT(wires=[0, 1])
            qml.CNOT(wires=[1, 2])
            qml.CNOT(wires=[2, 3])
            return qml.expval(self.H)
        
        opt = self.get_optimizer()
        params = np.random.randn(4, requires_grad=True)
        for i in range(100):
            params = opt.step(lambda p: circuit(p), params)
            energy = circuit(params)
            if i % 10 == 0:
                print(f"Step {i}: Energy = {energy}")
        return energy

# Executar o modelo para H2
test_H2 = H2Model(optimizer="COBYLA")
energy_H2 = test_H2.run_vqe()
print(f"Energia do estado fundamental do H2: {energy_H2:.6f} Ha")